In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
import pandas as pd
import numpy as np
import sklearn

In [3]:
import shap

In [5]:
train = pd.read_csv(r'C:\Users\Lenovo\OneDrive\Aakash\Codes\Autogluon\Data\train.csv')

In [6]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [8]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [15]:
train_red = train.drop(['PassengerId', 'Name', 'Ticket'], axis = 1)

In [16]:
train_red.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


In [9]:
test = pd.read_csv(r'C:\Users\Lenovo\OneDrive\Aakash\Codes\Autogluon\Data\test.csv')

In [24]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [26]:
test_result = pd.read_csv(r'C:\Users\Lenovo\OneDrive\Aakash\Codes\Autogluon\Data\gender_submission.csv')

In [29]:
test_result.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [30]:
test_merged = test.merge(test_result,on = 'PassengerId',how = 'inner')

In [31]:
test_merged.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1


In [19]:
%%time
predictor = TabularPredictor(label = 'Survived', eval_metric = 'log_loss', verbosity = 4).fit(train_red, ag_args_fit = {'num_gpus':1}, presets = 'best_quality', excluded_model_types = ['KNN'], holdout_frac = 0.3)

No path specified. Models will be saved in: "AutogluonModels/ag-20220115_133638\"
Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'ag_args_fit': {'num_gpus': 1},
 'auto_stack': True,
 'excluded_model_types': ['KNN'],
 'holdout_frac': 0.3}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': {'num_gpus': 1},
 'auto_stack': True,
 'calibrate': False,
 'excluded_model_types': ['KNN'],
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': 0.3,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': None,
 'num_bag_sets': None,
 'num_stack_levels': None,
 'pseudo_data': None,
 'quantile_levels': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 4}
Saving AutogluonModels/ag-20220115_133638\

		('float', [])     : 2 | ['Age', 'Fare']
		('int', [])       : 3 | ['Pclass', 'SibSp', 'Parch']
		('int', ['bool']) : 1 | ['Sex']
	0.1s = Fit runtime
	8 features in original data used to generate 8 features in processed data.
	Train Data (Processed) Memory Usage: 0.04 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.13s ...
AutoGluon will gauge predictive performance using evaluation metric: 'log_loss'
	This metric expects predicted probabilities rather than predicted class labels, so you'll need to use predict_proba() instead of predict()
	To change this, specify the eval_metric argument of fit()
Saving AutogluonModels/ag-20220115_133638\learner.pkl
Saving AutogluonModels/ag-20220115_133638\utils\data\X.pkl
Saving AutogluonModels/ag-20220115_133638\utils\data\y.pkl
AutoGluon will fit 2 stack levels (L1 to L2) ...
Excluded Model Types: ['KNN']
	Found 'KNN' model in hyperparameters, but 'KNN' is present in `excluded_model_types` and will be removed.


[1]	valid_set's binary_logloss: 0.649899
[2]	valid_set's binary_logloss: 0.635073
[3]	valid_set's binary_logloss: 0.62068
[4]	valid_set's binary_logloss: 0.60736
[5]	valid_set's binary_logloss: 0.596372
[6]	valid_set's binary_logloss: 0.587108
[7]	valid_set's binary_logloss: 0.576763
[8]	valid_set's binary_logloss: 0.570062
[9]	valid_set's binary_logloss: 0.562238
[10]	valid_set's binary_logloss: 0.555212
[11]	valid_set's binary_logloss: 0.548249
[12]	valid_set's binary_logloss: 0.542712
[13]	valid_set's binary_logloss: 0.537552
[14]	valid_set's binary_logloss: 0.531496
[15]	valid_set's binary_logloss: 0.526319
[16]	valid_set's binary_logloss: 0.521846
[17]	valid_set's binary_logloss: 0.51787
[18]	valid_set's binary_logloss: 0.514837
[19]	valid_set's binary_logloss: 0.51164
[20]	valid_set's binary_logloss: 0.507809
[21]	valid_set's binary_logloss: 0.505151
[22]	valid_set's binary_logloss: 0.502835
[23]	valid_set's binary_logloss: 0.501077
[24]	valid_set's binary_logloss: 0.499243
[25]	

[203]	valid_set's binary_logloss: 0.446193
[204]	valid_set's binary_logloss: 0.446304
[205]	valid_set's binary_logloss: 0.445707
[206]	valid_set's binary_logloss: 0.445762
[207]	valid_set's binary_logloss: 0.445562
[208]	valid_set's binary_logloss: 0.445468
[209]	valid_set's binary_logloss: 0.445371
[210]	valid_set's binary_logloss: 0.445414
[211]	valid_set's binary_logloss: 0.445879
[212]	valid_set's binary_logloss: 0.446319
[213]	valid_set's binary_logloss: 0.446608
[214]	valid_set's binary_logloss: 0.445774
[215]	valid_set's binary_logloss: 0.445889
[216]	valid_set's binary_logloss: 0.446111
[217]	valid_set's binary_logloss: 0.446767
[218]	valid_set's binary_logloss: 0.44669
[219]	valid_set's binary_logloss: 0.446701
[220]	valid_set's binary_logloss: 0.446285
[221]	valid_set's binary_logloss: 0.446293
[222]	valid_set's binary_logloss: 0.446057
[223]	valid_set's binary_logloss: 0.446056
[224]	valid_set's binary_logloss: 0.445982
[225]	valid_set's binary_logloss: 0.445023
[226]	valid_

[410]	valid_set's binary_logloss: 0.437254
[411]	valid_set's binary_logloss: 0.437141
[412]	valid_set's binary_logloss: 0.437845
[413]	valid_set's binary_logloss: 0.437882
[414]	valid_set's binary_logloss: 0.437573
[415]	valid_set's binary_logloss: 0.437444
[416]	valid_set's binary_logloss: 0.437122
[417]	valid_set's binary_logloss: 0.437281
[418]	valid_set's binary_logloss: 0.43687
[419]	valid_set's binary_logloss: 0.437185
[420]	valid_set's binary_logloss: 0.437158
[421]	valid_set's binary_logloss: 0.436999
[422]	valid_set's binary_logloss: 0.437322
[423]	valid_set's binary_logloss: 0.437585
[424]	valid_set's binary_logloss: 0.438237
[425]	valid_set's binary_logloss: 0.437884
[426]	valid_set's binary_logloss: 0.437868
[427]	valid_set's binary_logloss: 0.437774
[428]	valid_set's binary_logloss: 0.437578
[429]	valid_set's binary_logloss: 0.437728
[430]	valid_set's binary_logloss: 0.437816
[431]	valid_set's binary_logloss: 0.437312
[432]	valid_set's binary_logloss: 0.436917
[433]	valid_

...

[195]	valid_set's binary_logloss: 0.615981
[196]	valid_set's binary_logloss: 0.617813
[197]	valid_set's binary_logloss: 0.619659
[198]	valid_set's binary_logloss: 0.62186
[199]	valid_set's binary_logloss: 0.623837
[200]	valid_set's binary_logloss: 0.62451
[201]	valid_set's binary_logloss: 0.627153
[202]	valid_set's binary_logloss: 0.629115
[203]	valid_set's binary_logloss: 0.629739
[204]	valid_set's binary_logloss: 0.631847
[205]	valid_set's binary_logloss: 0.633977
[206]	valid_set's binary_logloss: 0.636098
[207]	valid_set's binary_logloss: 0.63799
[208]	valid_set's binary_logloss: 0.640724
[209]	valid_set's binary_logloss: 0.643383
[210]	valid_set's binary_logloss: 0.646301
[211]	valid_set's binary_logloss: 0.647987
[212]	valid_set's binary_logloss: 0.650843
[213]	valid_set's binary_logloss: 0.654367
[214]	valid_set's binary_logloss: 0.657116
[215]	valid_set's binary_logloss: 0.659067
[216]	valid_set's binary_logloss: 0.661251
[217]	valid_set's binary_logloss: 0.665576
[218]	valid_se

Saving AutogluonModels/ag-20220115_133638\models\LightGBMLarge_BAG_L2\utils\oof.pkl
Saving AutogluonModels/ag-20220115_133638\models\LightGBMLarge_BAG_L2\model.pkl
	-0.4214	 = Validation score   (log_loss)
	159.0s	 = Training   runtime
	0.05s	 = Validation runtime
Saving AutogluonModels/ag-20220115_133638\models\trainer.pkl
 predicted probabilities will be transformed to never =0 since eval_metric='log_loss'
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L2\utils\oof.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBM_BAG_L2\utils\oof.pkl
Loading: AutogluonModels/ag-20220115_133638\models\RandomForestGini_BAG_L2\utils\oof.pkl
Loading: AutogluonModels/ag-20220115_133638\models\RandomForestEntr_BAG_L2\utils\oof.pkl
Loading: AutogluonModels/ag-20220115_133638\models\ExtraTreesGini_BAG_L2\utils\oof.pkl
Loading: AutogluonModels/ag-20220115_133638\models\ExtraTreesEntr_BAG_L2\utils\oof.pkl
Loading: AutogluonModels/ag-20220115_133638\models\XGBoost_BAG_L2\utils\


[353]	valid_set's binary_logloss: 0.936683
[354]	valid_set's binary_logloss: 0.939107


	WeightedEnsemble_L3: 	{'ag_args': {'valid_base': False, 'name_bag_suffix': '', 'model_type': <class 'autogluon.core.models.greedy_ensemble.greedy_weighted_ensemble_model.GreedyWeightedEnsembleModel'>, 'priority': 0}, 'ag_args_ensemble': {'save_bag_folds': True}}
Fitting model: WeightedEnsemble_L3 ...
WeightedEnsemble predicted probabilities will be transformed to never =0 since eval_metric='log_loss'
	Dropped 0 of 8 features.
	Dropped 0 of 8 features.
Saving AutogluonModels/ag-20220115_133638\models\WeightedEnsemble_L3\utils\model_template.pkl
S1F1 predicted probabilities will be transformed to never =0 since eval_metric='log_loss'
	Dropped 0 of 8 features.
	Fitting S1F1 with 'num_gpus': 0, 'num_cpus': 8
Ensemble size: 73
Ensemble indices: [0, 4, 6, 5, 5, 1, 4, 6, 5, 0, 5, 7, 4, 5, 6, 5, 0, 4, 6, 5, 0, 4, 5, 6, 5, 1, 4, 0, 5, 6, 4, 7, 5, 0, 5, 4, 6, 5, 0, 4, 6, 5, 5, 6, 5, 0, 4, 7, 5, 4, 6, 0, 5, 4, 6, 5, 1, 5, 0, 4, 6, 5, 0, 5, 7, 4, 5, 6, 5, 4, 0, 6, 5]
Ensemble weights: 
[0.1643835

Wall time: 8min 41s


In [33]:
leaderboard_train = predictor.leaderboard(train_red, extra_metrics = ['roc_auc', 'log_loss'], extra_info = True)
leaderboard_train

Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L1\info.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L1\S1F1\info.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L1\S1F1\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L1\S1F2\info.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L1\S1F2\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L1\S1F3\info.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L1\S1F3\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L1\S1F4\info.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L1\S1F4\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L1\S1F5\info.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L1\S1F5\model.pkl
Loading:

Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L2\S1F4\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L2\S1F5\info.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L2\S1F5\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L2\S1F6\info.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L2\S1F6\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L2\S1F7\info.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L2\S1F7\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L2\S1F8\info.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L2\S1F8\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBM_BAG_L2\info.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBM_BAG_L2\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBM_BAG_L2\S1F1\info.pkl
Loading: Autog

Model scores:
{'ExtraTreesEntr_BAG_L1': -0.117578035754892, 'ExtraTreesGini_BAG_L1': -0.11779264079649475, 'LightGBMLarge_BAG_L1': -0.23010460775675864, 'LightGBMXT_BAG_L1': -0.3553955869333177, 'LightGBM_BAG_L1': -0.2924341175487781, 'RandomForestEntr_BAG_L1': -0.11615638342238242, 'RandomForestGini_BAG_L1': -0.11581343723127732, 'XGBoost_BAG_L1': -0.2887373321703254, 'ExtraTreesEntr_BAG_L2': -0.2759428547881335, 'ExtraTreesGini_BAG_L2': -0.2801557266859991, 'LightGBMLarge_BAG_L2': -0.3040251113248594, 'LightGBMXT_BAG_L2': -0.27233214351096674, 'LightGBM_BAG_L2': -0.2814847252150845, 'RandomForestEntr_BAG_L2': -0.2591057583045814, 'RandomForestGini_BAG_L2': -0.25664856150436954, 'WeightedEnsemble_L2': -0.2537329467860135, 'XGBoost_BAG_L2': -0.27998887137097006, 'WeightedEnsemble_L3': -0.2751928520285421}


                      model  score_test   roc_auc  log_loss  score_val  pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order  num_features  num_models  num_models_w_ancestors  memory_size  memory_size_w_ancestors  memory_size_min  memory_size_min_w_ancestors  num_ancestors  num_descendants             model_type             child_model_type                                    hyperparameters hyperparameters_fit                                        ag_args_fit                                           features                              child_hyperparameters child_hyperparameters_fit                                  child_ag_args_fit                                          ancestors                                        descendants
0   RandomForestGini_BAG_L1   -0.115813  0.997878 -0.115813  -0.557766        0.085666       0.098000    0.422695                 0.085666                0.098000

,model,score_test,roc_auc,log_loss,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,...,child_model_type,hyperparameters,hyperparameters_fit,ag_args_fit,features,child_hyperparameters,child_hyperparameters_fit,child_ag_args_fit,ancestors,descendants
0,RandomForestGini_BAG_L1,-0.115813,0.997878,-0.115813,-0.557766,0.085666,0.098000,0.422695,0.085666,0.098000,...,RFModel,"{'use_orig_features': True, 'max_base_models':...",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[Age, Embarked, Pclass, Cabin, Sex, SibSp, Far...","{'n_estimators': 300, 'n_jobs': 8, 'random_sta...",{'n_estimators': 300},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...",[],"[ExtraTreesEntr_BAG_L2, XGBoost_BAG_L2, ExtraT..."
1,RandomForestEntr_BAG_L1,-0.116156,0.997878,-0.116156,-0.574330,0.116266,0.097024,0.377851,0.116266,0.097024,...,RFModel,"{'use_orig_features': True, 'max_base_models':...",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[Age, Embarked, Pclass, Cabin, Sex, SibSp, Far...","{'n_estimators': 300, 'n_jobs': 8, 'random_sta...",{'n_estimators': 300},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...",[],"[ExtraTreesEntr_BAG_L2, XGBoost_BAG_L2, LightG..."
2,ExtraTreesEntr_BAG_L1,-0.117578,0.997766,-0.117578,-0.588626,0.072147,0.095721,0.375617,0.072147,0.095721,...,XTModel,"{'use_orig_features': True, 'max_base_models':...",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[Age, Embarked, Pclass, Cabin, Sex, SibSp, Far...","{'n_estimators': 300, 'n_jobs': 8, 'random_sta...",{'n_estimators': 300},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...",[],"[ExtraTreesEntr_BAG_L2, XGBoost_BAG_L2, ExtraT..."
3,ExtraTreesGini_BAG_L1,-0.117793,0.997750,-0.117793,-0.561316,0.083475,0.094958,0.364121,0.083475,0.094958,...,XTModel,"{'use_orig_features': True, 'max_base_models':...",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[Age, Embarked, Pclass, Cabin, Sex, SibSp, Far...","{'n_estimators': 300, 'n_jobs': 8, 'random_sta...",{'n_estimators': 300},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...",[],"[ExtraTreesEntr_BAG_L2, XGBoost_BAG_L2, LightG..."
4,LightGBMLarge_BAG_L1,-0.230105,0.990088,-0.230105,-0.423181,0.127975,0.074294,151.791832,0.127975,0.074294,...,LGBModel,"{'use_orig_features': True, 'max_base_models':...",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[Age, Embarked, Pclass, Cabin, Sex, SibSp, Far...","{'num_boost_round': 10000, 'num_threads': -1, ...",{'num_boost_round': 69},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...",[],"[ExtraTreesEntr_BAG_L2, XGBoost_BAG_L2, ExtraT..."
5,WeightedEnsemble_L2,-0.253733,0.977649,-0.253733,-0.403136,1.293731,0.433178,262.757876,0.015941,0.000992,...,GreedyWeightedEnsembleModel,"{'use_orig_features': False, 'max_base_models'...",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[XGBoost_BAG_L1, LightGBMLarge_BAG_L1, LightGB...",{'ensemble_size': 100},{'ensemble_size': 69},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[XGBoost_BAG_L1, LightGBMLarge_BAG_L1, LightGB...",[]
6,RandomForestGini_BAG_L2,-0.256649,0.959520,-0.256649,-0.436786,1.502159,0.725646,263.565676,0.068747,0.102782,...,RFModel,"{'use_orig_features': True, 'max_base_models':...",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[XGBoost_BAG_L1, LightGBMLarge_BAG_L1, Age, Ex...","{'n_estimators': 300, 'n_jobs': 8, 'random_sta...",{'n_estimators': 300},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[XGBoost_BAG_L1, LightGBMLarge_BAG_L1, ExtraTr...",[]
7,RandomForestEntr_BAG_L2,-0.259106,0.958628,-0.259106,-0.445423,1.550359,0.709687,263.563578,0.116947,0.086823,...,RFModel,"{'use_orig_features': True, 'max_base_models':...",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[XGBoost_BAG_L1, LightGBMLarge_BAG_L1, Age, Ex...","{'n_estimators': 300, 'n_jobs': 8, 'random_sta...",{'n_estimators': 300},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[XGBoost_BAG_L1, LightGBMLarge_BAG_L1, ExtraTr...",[]
8,LightGBMXT_BAG_

In [32]:
leaderboard_test = predictor.leaderboard(test_merged, extra_metrics = ['roc_auc', 'log_loss'], extra_info = False)
leaderboard_test

Loading: AutogluonModels/ag-20220115_133638\models\ExtraTreesEntr_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\ExtraTreesGini_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMLarge_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBM_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\RandomForestEntr_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\RandomForestGini_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\XGBoost_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\ExtraTreesEntr_BAG_L2\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\ExtraTreesGini_BAG_L2\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMLarge_BAG_L2\model.pkl
Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L2\model.pkl
Loading: AutogluonModel

                      model  score_test   roc_auc  log_loss  score_val  pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0         LightGBMXT_BAG_L1   -0.284406  0.969133 -0.284406  -0.432279        0.116600       0.046090   49.224906                 0.116600                0.046090          49.224906            1       True          1
1     ExtraTreesGini_BAG_L2   -0.314380  0.937042 -0.314380  -0.426730        1.460606       0.723629  263.514566                 0.068371                0.100765           0.362068            2       True         14
2           LightGBM_BAG_L1   -0.314842  0.947022 -0.314842  -0.409082        0.085361       0.055180   38.258141                 0.085361                0.055180          38.258141            1       True          2
3   RandomForestGini_BAG_L2   -0.318281  0.932727 -0.318281  -0.436786        1.461936       0.725646  263.565676                 0.

,model,score_test,roc_auc,log_loss,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L1,-0.284406,0.969133,-0.284406,-0.432279,0.116600,0.046090,49.224906,0.116600,0.046090,49.224906,1,True,1
1,ExtraTreesGini_BAG_L2,-0.314380,0.937042,-0.314380,-0.426730,1.460606,0.723629,263.514566,0.068371,0.100765,0.362068,2,True,14
2,LightGBM_BAG_L1,-0.314842,0.947022,-0.314842,-0.409082,0.085361,0.055180,38.258141,0.085361,0.055180,38.258141,1,True,2
3,RandomForestGini_BAG_L2,-0.318281,0.932727,-0.318281,-0.436786,1.461936,0.725646,263.565676,0.069701,0.102782,0.413178,2,True,12
4,LightGBMXT_BAG_L2,-0.318552,0.941630,-0.318552,-0.402651,1.477034,0.693335,292.007780,0.084799,0.070471,28.855282,2,True,10
5,WeightedEnsemble_L2,-0.319459,0.945142,-0.319459,-0.403136,1.241732,0.433178,262.757876,0.013551,0.000992,0.345117,2,True,9
6,WeightedEnsemble_L3,-0.324017,0.933988,-0.324017,-0.392414,2.537821,1.065281,514.048034,0.009414,0.000996,0.339418,3,True,18
7,XGBoost_BAG_L1,-0.326600,0.942743,-0.326600,-0.414015,0.726383,0.061599,22.337335,0.726383,0.061599,22.337335,1,True,7
8,RandomForestEntr_BAG_L2,-0.329264,0.926741,-0.329264,-0.445423,1.506778,0.709687,263.563578,0.114543,0.086823,0.411080,2,True,13
9,ExtraTreesEntr_BAG_L2,-0.329692,0.928868,-0.329692,-0.425080,1.512656,0.722339,263.536092,0.120421,0.099474,0.383594,2,True,15


In [34]:
pred = predictor.feature_importance(train_red, model = 'LightGBMXT_BAG_L1')

Loading: AutogluonModels/ag-20220115_133638\models\LightGBMXT_BAG_L1\model.pkl
Computing feature importance via permutation shuffling for 8 features using 891 rows with 3 shuffle sets...
	3.76s	= Expected runtime (1.25s per shuffle set)
	1.26s	= Actual runtime (Completed 3 of 3 shuffle sets)


In [35]:
pred

,importance,stddev,p_value,n,p99_high,p99_low
Sex,0.343802,0.023141,0.000753,3,0.476400,0.211204
Age,0.109759,0.003250,0.000146,3,0.128384,0.091134
Fare,0.060855,0.002369,0.000252,3,0.074431,0.047280
SibSp,0.046263,0.001968,0.000301,3,0.057542,0.034985
Embarked,0.029647,0.005540,0.005719,3,0.061389,-0.002095
Cabin,0.021413,0.001873,0.001271,3,0.032148,0.010679
Pclass,0.016869,0.004633,0.012118,3,0.043416,-0.009679
Parch,0.015548,0.002114,0.003053,3,0.027661,0.003435


In [ ]:
pred.to_csv('train_feature_importance.csv')

In [36]:
predictor.delete_models(models_to_keep = 'LightGBMXT_BAG_L1', dry_run = False)

Loading: AutogluonModels/ag-20220115_133638\models\LightGBM_BAG_L1\model.pkl
Deleting model LightGBM_BAG_L1. All files under AutogluonModels/ag-20220115_133638\models\LightGBM_BAG_L1\ will be removed.
Loading: AutogluonModels/ag-20220115_133638\models\RandomForestGini_BAG_L1\model.pkl
Deleting model RandomForestGini_BAG_L1. All files under AutogluonModels/ag-20220115_133638\models\RandomForestGini_BAG_L1\ will be removed.
Loading: AutogluonModels/ag-20220115_133638\models\RandomForestEntr_BAG_L1\model.pkl
Deleting model RandomForestEntr_BAG_L1. All files under AutogluonModels/ag-20220115_133638\models\RandomForestEntr_BAG_L1\ will be removed.
Loading: AutogluonModels/ag-20220115_133638\models\ExtraTreesGini_BAG_L1\model.pkl
Deleting model ExtraTreesGini_BAG_L1. All files under AutogluonModels/ag-20220115_133638\models\ExtraTreesGini_BAG_L1\ will be removed.
Loading: AutogluonModels/ag-20220115_133638\models\ExtraTreesEntr_BAG_L1\model.pkl
Deleting model ExtraTreesEntr_BAG_L1. All files

In [38]:
predictor = TabularPredictor.load(r'C:\Users\Lenovo\OneDrive\Aakash\Codes\Autogluon\AutogluonModels\ag-20220115_133638')

Loading: C:\Users\Lenovo\OneDrive\Aakash\Codes\Autogluon\AutogluonModels\ag-20220115_133638\predictor.pkl
Loading: C:\Users\Lenovo\OneDrive\Aakash\Codes\Autogluon\AutogluonModels\ag-20220115_133638\learner.pkl
Loading: C:\Users\Lenovo\OneDrive\Aakash\Codes\Autogluon\AutogluonModels\ag-20220115_133638\models\trainer.pkl


In [39]:
train_red['Prediction'] = np.array(predictor.predict_proba(train_red))[:,1]

Loading: C:\Users\Lenovo\OneDrive\Aakash\Codes\Autogluon\AutogluonModels\ag-20220115_133638\models\LightGBMXT_BAG_L1\model.pkl


In [40]:
train_red.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Prediction
0,0,3,male,22.0,1,0,7.2500,NaN,S,0.090598
1,1,1,female,38.0,1,0,71.2833,C85,C,0.971431
2,1,3,female,26.0,0,0,7.9250,NaN,S,0.577133
3,1,1,female,35.0,1,0,53.1000,C123,S,0.952453
4,0,3,male,35.0,0,0,8.0500,NaN,S,0.109135


In [42]:
print(sklearn.metrics.roc_auc_score(train_red['Survived'], train_red['Prediction']))

0.9200779727095516


In [43]:
test_merged['Prediction'] = np.array(predictor.predict_proba(test_merged))[:,1]

Loading: C:\Users\Lenovo\OneDrive\Aakash\Codes\Autogluon\AutogluonModels\ag-20220115_133638\models\LightGBMXT_BAG_L1\model.pkl


In [44]:
print(sklearn.metrics.roc_auc_score(test_merged['Survived'], test_merged['Prediction']))

0.9691333597150772
